In [ ]:
import argparse
import os
import sys 
sys.path.insert(0, '/raven/u/noka/VAEEG') 
from src.model.opts.dataset import ClipDataset
from src.model.net.modelA import VAEEG, re_parameterize
import yaml
import torch
import numpy as np
from collections import OrderedDict
from torch.utils.data import DataLoader
import torch.nn as nn
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.spatial.distance import cdist, pdist
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    confusion_matrix, roc_auc_score, RocCurveDisplay
)
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'torch'

In [2]:
def load_model(model: nn.Module, ckpt_file: str):
    """
    Loads a checkpoint saved from single- or multi-GPU training into a model
    running on a single GPU or CPU. Returns any auxiliary info found.
    """
    
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    ckpt = torch.load(ckpt_file, map_location=device)
    sd = ckpt["model"]

    # Remove "module.", if saved under DataParallel/DDP"
    if next(iter(sd)).startswith("module."):
        sd = {k[len("module."):]: v for k, v in sd.items()}
    model.load_state_dict(sd)
    model.to(device)
    
    return device
def init_model(in_channels, z_dim, deterministic, ckpt_file, negative_slope=0.2, decoder_last_lstm=False): 
    model = VAEEG(in_channels=in_channels,
                    z_dim=z_dim,
                    negative_slope=negative_slope,
                    decoder_last_lstm=decoder_last_lstm,
                    deterministic=deterministic)
    device = load_model(model, ckpt_file=ckpt_file)
    return model, device
def init_dl(data_root, band, batch_size=1024, num_workers=1, shuffle=False): 
    ds = ClipDataset(
        data_dir=data_root,
        band_name=band,
        clip_len=256
    )
    loader = DataLoader(
        ds,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=shuffle,
    )
    return loader
def get_latent(model, loader, device, z_dim):
    model.eval()
    num_samples = len(loader.dataset)
    z_arr = np.zeros((num_samples, z_dim), dtype=np.float32)  
    idx = 0
    encoder = model.module.encoder if hasattr(model, 'module') else model.encoder
    # Get the deterministic setting from the model
    deterministic = model.module.deterministic if hasattr(model, 'module') else model.deterministic
    with torch.no_grad():
        for x in loader:
            x = x.to(device)
            mu, log_var = encoder(x)
            if deterministic:
                z_batch = mu  # In deterministic mode, log_var is None, so just use mu
            else:
                z_batch = re_parameterize(mu, log_var, deterministic=deterministic)
            bs = z_batch.size(0)
            z_arr[idx:idx + bs, :] = z_batch.cpu().numpy()
            idx += bs
    return z_arr

In [3]:
# Dictionaries for latents
ae_latents = {} 
vae_latents = {}
pca_latents = {}
pca_train = {}
pca_test = {}

In [8]:
# Define model parameters
bands = ["delta", "theta", "alpha", "low_beta", "high_beta"]
z_dim = 32
epoch = 37
in_channels=1
seed = 42 
deterministic = False
if deterministic: 
    ckpt_root = "/raven/u/noka/VAEEG/models_new/vanilla"
else: 
    ckpt_root = "/raven/u/noka/VAEEG/models_new/variational"
data_dir = "/ptmp/noka/new_data"

In [5]:
# Prepare data for PCA fitting
rng_test = np.random.RandomState(seed)
starts_test = rng_test.randint(0, 1280 - 256 + 1, size=288182)
rng_train = np.random.RandomState(seed)
starts_train = rng_train.randint(0, 1280 - 256 + 1, size=2575060)
for band in tqdm(bands): 
    band_train = np.load(os.path.join(data_dir, "train", f"{band}.npy"))
    band_test = np.load(os.path.join(data_dir, "test", f"{band}.npy"))
    orig_train = np.zeros((band_train.shape[0], 256), dtype=band_train.dtype)
    orig_test = np.zeros((band_test.shape[0], 256), dtype=band_test.dtype)
    for i in range(band_train.shape[0]):
        orig_train[i] = band_train[i, starts_train[i]:starts_train[i] + 256]
    for i in range(band_test.shape[0]):
        orig_test[i] = band_test[i, starts_test[i]:starts_test[i] + 256]
    pca_train[band] = orig_train
    pca_test[band] = orig_test

100%|██████████| 5/5 [02:00<00:00, 24.05s/it]


In [9]:
# Get AE and VAE latents
for band in tqdm(bands):
    ckpt_file = os.path.join(ckpt_root, band, f"z{z_dim}", f"ckpt_epoch_{epoch}.ckpt")
    model, device = init_model(in_channels=in_channels, z_dim=z_dim, deterministic=deterministic, ckpt_file=ckpt_file)
    loader = init_dl(os.path.join(data_dir, "test"), band)
    latent = get_latent(model=model, loader=loader, device=device, z_dim=z_dim)
    if deterministic: 
        ae_latents[band] = latent
    else: 
        vae_latents[band] = latent

100%|██████████| 5/5 [00:16<00:00,  3.29s/it]


In [7]:
# Get PCA latents
for band in tqdm(bands):
    data_train = pca_train[band]   
    pca = PCA(n_components=50, random_state=42)
    pca.fit(data_train)
    print("Explained variance ratio sum:", np.sum(pca.explained_variance_ratio_))
    data_test = pca_test[band]     
    latents = pca.transform(data_test)
    pca_latents[band] = latents

 20%|██        | 1/5 [00:00<00:02,  1.41it/s]

Explained variance ratio sum: 0.99999994


 40%|████      | 2/5 [00:01<00:01,  1.51it/s]

Explained variance ratio sum: 1.0


 60%|██████    | 3/5 [00:01<00:01,  1.54it/s]

Explained variance ratio sum: 0.9999999


 80%|████████  | 4/5 [00:02<00:00,  1.56it/s]

Explained variance ratio sum: 0.9999999


100%|██████████| 5/5 [00:03<00:00,  1.55it/s]

Explained variance ratio sum: 1.0


In [19]:
lr_vae = {}
lr_ae = {}
lr_pca = {}

In [20]:
knn_vae = {}
knn_ae = {}
knn_pca = {}

In [19]:
X = vae_latents["alpha"]
y = np.load("/ptmp/noka/labels/test.npy")

classes = np.unique(y)
n_features = X.shape[1]

mu = np.mean(X, axis=0)

S_W = np.zeros((n_features, n_features))
S_B = np.zeros((n_features, n_features))

for c in classes:
    X_c = X[y == c]
    mu_c = np.mean(X_c, axis=0)
    n_c = X_c.shape[0]

    # Within-class scatter
    S_W += (X_c - mu_c).T @ (X_c - mu_c)

    # Between-class scatter
    mean_diff = (mu_c - mu).reshape(-1, 1)
    S_B += n_c * (mean_diff @ mean_diff.T)

# Natural scatter ratio
scatter_ratio = np.trace(S_B) / np.trace(S_W)
print(f"Natural scatter ratio (trace(S_B)/trace(S_W)): {scatter_ratio:.4f}")

Natural scatter ratio (trace(S_B)/trace(S_W)): 0.0024


In [18]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

X = vae_latents["alpha"]        
y = np.load("/ptmp/noka/labels/test.npy")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=42
)

lda = LinearDiscriminantAnalysis(n_components=1, priors=[0.9, 0.1])  
lda.fit(X_train, y_train)

y_pred = lda.predict(X_test)                
y_proba = lda.predict_proba(X_test)[:, 1]   

# --- Basic metrics ---
acc  = accuracy_score(y_test, y_pred)
auc  = roc_auc_score(y_test, y_proba)
cm   = confusion_matrix(y_test, y_pred)
print(f"Accuracy: {acc:.3f}  |  ROC-AUC: {auc:.3f}")
print("Confusion matrix:\n", cm)

Accuracy: 0.851  |  ROC-AUC: 0.727
Confusion matrix:
 [[61289   485]
 [10229    43]]


In [22]:
y = np.load("/ptmp/noka/labels/test.npy")
for band in bands: 
    X = ae_latents[band]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.25, random_state=42, stratify=y
    )

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test  = scaler.transform(X_test)
    # --- Logistic Regression ---
    logreg = LogisticRegression(
        random_state=42, 
        class_weight="balanced", 
        solver="saga", 
        max_iter=200,
        n_jobs=-1
    )
    logreg.fit(X_train, y_train)

    # Predictions and probabilities
    y_pred  = logreg.predict(X_test)
    y_proba = logreg.predict_proba(X_test)[:, 1] 
    auc  = roc_auc_score(y_test, y_proba)
    lr_ae[band] = auc, y_proba

In [28]:
lr_ae["theta"][0]

0.7446073795703099

In [39]:
from MLstatkit import Delong_test

for band in bands: 
    z, p, ci_A, ci_B, auc_A, auc_B, info = Delong_test(
        y_test, lr_auc_ae[band][1], lr_auc_pca[band][1],
        alpha=0.95, return_ci=True, return_auc=True, verbose=1
    )

    def stars_from_p(p):
        if p < 0.001:
            return "***"
        elif p < 0.01:
            return "**"
        elif p < 0.05:
            return "*"
        else:
            return ""
    print(f"z-score  : {z:.4f}, p-value = {p:.3e} {stars_from_p(p)}")

=== DeLong Test ===
Samples: total=72046, pos=10272, neg=61774
Tie rate: A=0.005, B=0.196
Step 1) Attempt standard DeLong.
[delong] z = -36.077465, p = 5.118e-285
z-score  : -36.0775, p-value = 5.118e-285 ***
=== DeLong Test ===
Samples: total=72046, pos=10272, neg=61774
Tie rate: A=0.004, B=0.145
Step 1) Attempt standard DeLong.
[delong] z = -57.212027, p = 0.000e+00
z-score  : -57.2120, p-value = 0.000e+00 ***
=== DeLong Test ===
Samples: total=72046, pos=10272, neg=61774
Tie rate: A=0.004, B=0.138
Step 1) Attempt standard DeLong.
[delong] z = -52.771932, p = 0.000e+00
z-score  : -52.7719, p-value = 0.000e+00 ***
=== DeLong Test ===
Samples: total=72046, pos=10272, neg=61774
Tie rate: A=0.008, B=0.183
Step 1) Attempt standard DeLong.
[delong] z = -47.872723, p = 0.000e+00
z-score  : -47.8727, p-value = 0.000e+00 ***
=== DeLong Test ===
Samples: total=72046, pos=10272, neg=61774
Tie rate: A=0.007, B=0.240
Step 1) Attempt standard DeLong.
[delong] z = -39.169505, p = 0.000e+00
z-score 

In [65]:
rng = np.random.default_rng(42)
idx_seiz = np.where(y == 1)[0]
idx_bkgd = np.where(y == 0)[0]
n_seiz   = len(idx_seiz)

idx_bkgd_down = rng.choice(idx_bkgd, size=n_seiz, replace=False)
idx_bal = np.concatenate([idx_seiz, idx_bkgd_down])
rng.shuffle(idx_bal)
for band in bands:
    X = raws[band]
    X_bal = X[idx_bal]
    y_bal = y[idx_bal]
    X_train, X_test, y_train, y_test = train_test_split(
        X_bal, y_bal, test_size=0.25, random_state=42, stratify=y_bal
    )

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test  = scaler.transform(X_test)
    k = 287  
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)

    y_proba = knn.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_proba)
    knn_raw[band] = (auc, y_proba)

In [80]:
for band, (auc, y_proba) in knn_pca.items():
    print(f"{band}: {auc:.3f}")

delta: 0.710
theta: 0.747
alpha: 0.740
low_beta: 0.718
high_beta: 0.674


In [62]:
# load raw
raws = {}
for band in bands: 
    raw = np.load(os.path.join(data_dir, f"test/{band}.npy"))
    raws[band] = raw 

In [73]:
len(knn_vae[band][1])

20544

In [67]:
from MLstatkit import Delong_test

for band in bands: 
    z, p, ci_A, ci_B, auc_A, auc_B, info = Delong_test(
        y_test, lr_auc_vae[band][1][idx_bal], knn_vae[band][1],
        alpha=0.95, return_ci=True, return_auc=True, verbose=1
    )

    def stars_from_p(p):
        if p < 0.001:
            return "***"
        elif p < 0.01:
            return "**"
        elif p < 0.05:
            return "*"
        else:
            return ""
    print(f"z-score  : {z:.4f}, p-value = {p:.3e} {stars_from_p(p)}")

ValueError: true, prob_A, prob_B must have the same length.